I couldn't quickly find a way to download google drive files with `wget`/`curl`, but the python code below seems to work correctly.

In [1]:
### Download the tarball

import os
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


file_id = '1YcygCgwlZiqAN92JWHnMMVColle5b5Wz'
destination = './daily_images.tar.gz'
if not os.path.exists(destination):
    download_file_from_google_drive(file_id, destination)

In [2]:
### Prepare the data

import os
import requests


# Extract tarball
!tar xfz daily_images.tar.gz

# Delete tarball
os.remove('daily_images.tar.gz')
    
# Prepare output directory for registered frames
output_frame_dir = 'out_frames'    
if not os.path.exists(output_frame_dir):
    os.makedirs(output_frame_dir)    

In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import rescale
from skimage.util import img_as_ubyte
import glob
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [4]:
import networkx as nx
from glob import glob

im_fns = sorted(glob('in_frames/*.jpg'))
im_fns = im_fns[2:]  # first 2 pics are no good

# im_fns = im_fns[:100]

In [5]:
from skimage.color import rgb2gray
from skimage.filters import gaussian, scharr
from skimage.exposure import is_low_contrast
from skimage.util import img_as_ubyte
from skimage.filters.rank import median
from skimage.morphology import disk


    
def process_image(im):
    """Transform RGB image to grayscale image suitable for registration"""
    gray = rgb2gray(im)
    gray = median(gray, disk(2))
    gradient = scharr(gray)
    soft_grad = gradient.astype(np.float32)**0.5
    return soft_grad


def match_images(ref_im, im, eps=5e-5, its=200):
    """Compute ECC registration on 2D images"""
    
    im_8bit = img_as_ubyte(im)
    ref_im_8bit = img_as_ubyte(ref_im)
    affine_trans = cv2.estimateRigidTransform(im_8bit, ref_im_8bit, fullAffine=True)
    if affine_trans is not None:
        aligned_im = cv2.warpAffine(im, affine_trans, (im.shape[1], im.shape[0]))
        affine_trans_homo = np.vstack([affine_trans, [0, 0, 1]])
    else:
        aligned_im = im
        affine_trans_homo = np.eye(3)
    
    warp_mode = cv2.MOTION_HOMOGRAPHY
    warp_matrix = np.eye(3, 3, dtype=np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, its,  eps)
    cc, warp_matrix = cv2.findTransformECC (aligned_im, ref_im, warp_matrix, warp_mode, criteria)
    
    return cc, np.dot(affine_trans_homo, warp_matrix)


def is_good_match(cc, warp_matrix):
    return cc > 0.6


def warp_image(im, matrix):
    """Apply perspective transform (3x3) to an image"""
    return cv2.warpPerspective (im, matrix, (im.shape[1], im.shape[0]))


graph = nx.DiGraph()
graph.add_nodes_from(im_fns)


for ref_im_num, ref_im_fn in enumerate(graph.nodes()):
    
    # process the ref image for matching
    ref_im = plt.imread(ref_im_fn)    
    processed_ref_im = process_image(ref_im)
        
    # try to match this frame to the next few frames
    for im_fn in graph.nodes()[ref_im_num+1:ref_im_num+3]:
        im = plt.imread(im_fn)
        processed_im = process_image(im)
        
        # Try to match this image to the reference        
        cc, warp_matrix = match_images(processed_ref_im, processed_im)
        inv_warp_matrix = np.linalg.inv(warp_matrix)
        
        # Validate transform
        if is_good_match(cc, warp_matrix):
            graph.add_edge(ref_im_fn, im_fn, warp_matrix=warp_matrix, cc=cc)
            graph.add_edge(im_fn, ref_im_fn, warp_matrix=inv_warp_matrix, cc=cc)
            print('matched %s to %s: %f' % (os.path.basename(ref_im_fn), os.path.basename(im_fn), cc))            
            break

# print(nx.info(graph))
# nx.is_strongly_connected(graph)

matched 003.jpg to 004.jpg: 0.866894
matched 004.jpg to 005.jpg: 0.862834
matched 005.jpg to 006.jpg: 0.763016
matched 006.jpg to 007.jpg: 0.826382
matched 007.jpg to 008.jpg: 0.783257
matched 008.jpg to 009.jpg: 0.811186
matched 009.jpg to 010.jpg: 0.888445
matched 010.jpg to 011.jpg: 0.917308
matched 011.jpg to 012.jpg: 0.909109
matched 012.jpg to 013.jpg: 0.937273
matched 013.jpg to 014.jpg: 0.826233
matched 014.jpg to 016.jpg: 0.948399
matched 015.jpg to 017.jpg: 0.716907
matched 016.jpg to 017.jpg: 0.776059
matched 017.jpg to 018.jpg: 0.794947
matched 018.jpg to 019.jpg: 0.883258
matched 019.jpg to 020.jpg: 0.721849
matched 020.jpg to 021.jpg: 0.774185
matched 021.jpg to 022.jpg: 0.803495
matched 022.jpg to 023.jpg: 0.885024
matched 023.jpg to 024.jpg: 0.844215
matched 024.jpg to 025.jpg: 0.857207
matched 025.jpg to 026.jpg: 0.811942
matched 026.jpg to 027.jpg: 0.874121
matched 027.jpg to 028.jpg: 0.928596
matched 028.jpg to 029.jpg: 0.811626
matched 029.jpg to 030.jpg: 0.923122
m

matched 235.jpg to 236.jpg: 0.795870
matched 236.jpg to 237.jpg: 0.667125
matched 237.jpg to 238.jpg: 0.766956
matched 238.jpg to 239.jpg: 0.942955
matched 239.jpg to 240.jpg: 0.910833
matched 240.jpg to 241.jpg: 0.842580
matched 241.jpg to 242.jpg: 0.880219
matched 242.jpg to 243.jpg: 0.924641
matched 243.jpg to 244.jpg: 0.870715
matched 244.jpg to 245.jpg: 0.874541
matched 245.jpg to 246.jpg: 0.761536
matched 246.jpg to 247.jpg: 0.906416
matched 247.jpg to 248.jpg: 0.920289
matched 248.jpg to 249.jpg: 0.854372
matched 249.jpg to 250.jpg: 0.745307
matched 250.jpg to 251.jpg: 0.868625
matched 251.jpg to 252.jpg: 0.728104
matched 252.jpg to 253.jpg: 0.726521
matched 253.jpg to 254.jpg: 0.766624
matched 254.jpg to 256.jpg: 0.915291
matched 256.jpg to 259.jpg: 0.819030
matched 259.jpg to 260.jpg: 0.816622
matched 260.jpg to 261.jpg: 0.833320
matched 261.jpg to 262.jpg: 0.766008
matched 262.jpg to 263.jpg: 0.753559
matched 263.jpg to 264.jpg: 0.886027
matched 264.jpg to 266.jpg: 0.912978
m

In [6]:
import shutil


def chain_transforms(graph, start_node, end_node):
    transform_to_end = np.eye(3, dtype=np.float32)
    path_nodes = nx.shortest_path(graph, start_node, end_node)
    for node_num, (node1, node2) in enumerate(zip(path_nodes[:-1], path_nodes[1:])):
        transform = graph[node2][node1]['warp_matrix']
        transform_to_end = np.dot(transform_to_end, transform)
        
        # debug
        if node_num == 0:
            cc = graph[node1][node2]['cc']        
            print(node1, node2, cc)
            
    return transform_to_end
                


out_dir = 'out_frames'
if not os.path.exists(out_dir):
    shutil.rmtree(out_dir)
    os.makedirs(out_dir)

    
ref_node = 'in_frames/101.jpg'
# ref_node = 'in_frames/055.jpg'


for im_num, node in enumerate(graph.nodes()):
    if node == ref_node:
        out_im = plt.imread(node)
        cc = 1.
    else:
        transform_to_ref = chain_transforms(graph, node, ref_node)
        im = plt.imread(node)
        out_im = warp_image(im, transform_to_ref)
        
    out_fn = os.path.join(out_dir, '%03d.png' % im_num)
    print(out_fn)
    plt.imsave(out_fn, out_im)
    

in_frames/003.jpg in_frames/004.jpg 0.8668942429941445
out_frames/000.png
in_frames/004.jpg in_frames/005.jpg 0.8628341550198434
out_frames/001.png
in_frames/005.jpg in_frames/006.jpg 0.7630158833643126
out_frames/002.png
in_frames/006.jpg in_frames/007.jpg 0.8263820033995534
out_frames/003.png
in_frames/007.jpg in_frames/008.jpg 0.7832572867647295
out_frames/004.png
in_frames/008.jpg in_frames/009.jpg 0.8111863966790358
out_frames/005.png
in_frames/009.jpg in_frames/010.jpg 0.88844493428558
out_frames/006.png
in_frames/010.jpg in_frames/011.jpg 0.9173083419227109
out_frames/007.png
in_frames/011.jpg in_frames/012.jpg 0.9091088212997083
out_frames/008.png
in_frames/012.jpg in_frames/013.jpg 0.9372726306501928
out_frames/009.png
in_frames/013.jpg in_frames/014.jpg 0.826233205621171
out_frames/010.png
in_frames/014.jpg in_frames/016.jpg 0.9483990568711282
out_frames/011.png
in_frames/015.jpg in_frames/017.jpg 0.7169072386210597
out_frames/012.png
in_frames/016.jpg in_frames/017.jpg 0.776

in_frames/118.jpg in_frames/117.jpg 0.9488152440382632
out_frames/112.png
in_frames/119.jpg in_frames/118.jpg 0.6548133889989157
out_frames/113.png
in_frames/120.jpg in_frames/119.jpg 0.7427945017901025
out_frames/114.png
in_frames/121.jpg in_frames/120.jpg 0.9492124070247835
out_frames/115.png
in_frames/122.jpg in_frames/121.jpg 0.9611660381394342
out_frames/116.png
in_frames/123.jpg in_frames/122.jpg 0.9752702644583346
out_frames/117.png
in_frames/124.jpg in_frames/123.jpg 0.9145527736154959
out_frames/118.png
in_frames/125.jpg in_frames/124.jpg 0.9147690439615489
out_frames/119.png
in_frames/126.jpg in_frames/125.jpg 0.7801560564967294
out_frames/120.png
in_frames/127.jpg in_frames/126.jpg 0.7514430256843012
out_frames/121.png
in_frames/128.jpg in_frames/127.jpg 0.6379936983311973
out_frames/122.png
in_frames/129.jpg in_frames/128.jpg 0.7428475476870976
out_frames/123.png
in_frames/130.jpg in_frames/129.jpg 0.787824416077998
out_frames/124.png
in_frames/131.jpg in_frames/130.jpg 0.8

in_frames/236.jpg in_frames/235.jpg 0.79586965214689
out_frames/223.png
in_frames/237.jpg in_frames/236.jpg 0.6671248893616536
out_frames/224.png
in_frames/238.jpg in_frames/237.jpg 0.7669559879199596
out_frames/225.png
in_frames/239.jpg in_frames/238.jpg 0.9429548991756105
out_frames/226.png
in_frames/240.jpg in_frames/239.jpg 0.9108332126890624
out_frames/227.png
in_frames/241.jpg in_frames/240.jpg 0.8425799456915255
out_frames/228.png
in_frames/242.jpg in_frames/241.jpg 0.8802193728066946
out_frames/229.png
in_frames/243.jpg in_frames/242.jpg 0.9246409654914468
out_frames/230.png
in_frames/244.jpg in_frames/243.jpg 0.8707145027691761
out_frames/231.png
in_frames/245.jpg in_frames/244.jpg 0.8745406430385054
out_frames/232.png
in_frames/246.jpg in_frames/245.jpg 0.7615361826276096
out_frames/233.png
in_frames/247.jpg in_frames/246.jpg 0.9064160327025886
out_frames/234.png
in_frames/248.jpg in_frames/247.jpg 0.9202886573509144
out_frames/235.png
in_frames/249.jpg in_frames/248.jpg 0.85

In [7]:
!echo y|ffmpeg -framerate 7 -f image2 -i in_frames/%*.jpg -c:v h264 -crf 1 original.mov
!echo y|ffmpeg -framerate 7 -f image2 -i out_frames/%*.png -c:v h264 -crf 1 corrected.mov

ffmpeg version 2.8.11-0ubuntu0.16.04.1 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.4) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --e

[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (ce)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (cf)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (c5)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (cb)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (ca)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (c5)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (cf)
[mjpeg @ 0x189eea0] Can not process SOS before SOF, skipping
[mjpeg @ 0x189eea0] bits 43 is invalid
Error while decoding stream #0:0: Invalid data found when processing input
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (c9)
[mjpeg @ 0x189eea0] invalid id 233
Error while decoding stream #0:0: Invalid data found when processing input
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (c7)
[mjpeg @ 0x189eea0] mjpeg: unsupported coding type (cf)
[mjpeg @ 0x189eea0] bits 148 is invalid
Error while decoding stream #0:0: Invalid data found when processing input
[mjpeg @ 0x189eea0] mjpeg: unsupported c

frame=  213 fps= 19 q=-1.0 Lsize=   57744kB time=00:00:30.14 bitrate=15693.3kbits/s    
video:57742kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.004149%
[libx264 @ 0x18a0ba0] frame I:14    Avg QP: 1.19  size:297138
[libx264 @ 0x18a0ba0] frame P:188   Avg QP: 2.84  size:277315
[libx264 @ 0x18a0ba0] frame B:11    Avg QP: 3.16  size:257438
[libx264 @ 0x18a0ba0] consecutive B-frames: 89.7% 10.3%  0.0%  0.0%
[libx264 @ 0x18a0ba0] mb I  I16..4: 21.3% 33.7% 45.0%
[libx264 @ 0x18a0ba0] mb P  I16..4: 14.7% 31.4% 30.7%  P16..4:  9.6%  7.5%  5.0%  0.0%  0.0%    skip: 1.2%
[libx264 @ 0x18a0ba0] mb B  I16..4:  6.6% 22.7% 23.8%  B16..8: 13.2%  6.2%  6.4%  direct: 9.3%  skip:11.7%  L0:35.4% L1:29.8% BI:34.8%
[libx264 @ 0x18a0ba0] 8x8 transform intra:40.4% inter:40.6%
[libx264 @ 0x18a0ba0] coded y,uvDC,uvAC intra: 91.1% 89.5% 85.6% inter: 91.6% 91.8% 91.7%
[libx264 @ 0x18a0ba0] i16 v,h,dc,p: 47% 15% 15% 23%
[libx264 @ 0x18a0ba0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 26% 29